## NLP12. 멋진 챗봇 만들기

### 필요한 모듈 설치와 임포트

In [ ]:
pip install sentencepiece

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
pip install --upgrade tensorflow==2.10.0

In [ ]:
import numpy 
import pandas as pd
import tensorflow as tf
import nltk
import gensim
from konlpy.tag import Mecab
import os
import re
import numpy as np
from collections import Counter
import tensorflow as tf
import sentencepiece as spm
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import random
import math
from tqdm import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

print(numpy.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__)

2.10.0
1.21.6
1.3.5
2.10.0
3.7
3.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 파일 디렉토리 설정과 데이터 불러오기

In [ ]:
workspace_path = os.getenv('HOME')+'/aiffel/transformer_chatbot'
data_path = os.getenv('HOME')+'/aiffel/transformer_chatbot/data'

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ChatbotData.csv')
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
df = pd.DataFrame(data)
questions = df['Q']; answers = df['A']
questions.head()

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

In [ ]:
questions = questions.values.tolist(); answers = answers.values.tolist()

In [ ]:
print(len(questions),len(answers))

11823 11823


In [ ]:
len(list(set(zip(questions, answers))))

11750

In [ ]:
def preprocess_sentence(sentence):
    
    sentence = sentence.strip()
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()
    
    return sentence    

In [ ]:
def tokenize(corpus, vocab_size=30000):
    mecab = Mecab()
    morph = [" ".join(mecab.morphs(sen)) for sen in corpus]
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',
                                                     num_words=vocab_size)
    tokenizer.fit_on_texts(morph)

    tensor = tokenizer.texts_to_sequences(morph)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

        
    return tensor, tokenizer, morph

In [ ]:
def build_corpus(questions, answers) :
    
 mecab = Mecab()
 lsz = list(set(zip(questions, answers)))
 que_attr = []; ans_attr = []
 que_tensor = []; ans_tensor = []
 for i in range(len(lsz)): 
    QA = list(lsz[i])
    if(len(QA[0])<=60 or len(QA[1])<=60) :
      q_tensor,_,q_morph = tokenize(preprocess_sentence(QA[0]))
      que_tensor.append(q_tensor); que_attr.append(q_morph) 
      a_tensor,_,a_morph = tokenize(preprocess_sentence(QA[1]))
      ans_tensor.append(a_tensor); ans_attr.append(a_morph) 

 return que_tensor, que_attr, ans_tensor, ans_attr           

In [ ]:
_,que_corpus,_,ans_corpus = build_corpus(questions, answers)

In [ ]:
que_corpus[0]

['오',
 '늘',
 '도',
 '',
 '연',
 '락',
 '하',
 '고',
 '싶',
 '은',
 '거',
 '',
 '꾹',
 '.',
 '꾹',
 '',
 '정',
 '말',
 '',
 '참',
 '았',
 '습',
 '니',
 '다',
 '.']

### 단어 부풀릴 목적의 함수 정의와 단어 부풀리기

In [ ]:
def lexical_sub(sentence, word2vec):
    import random

    res = ""

    try:
        _from = random.choice(sentence)
        _to = wv_model.wv.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in sentence :
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [ ]:
from tqdm.notebook import tqdm

def augment_corpus(src_corpus, tgt_corpus, wv):
    new_src_corpus = []
    new_tgt_corpus = []
    corpus_size = len(src_corpus)
    
    for i in tqdm(range(corpus_size)):
        old_src = src_corpus[i]
        old_tgt = tgt_corpus[i]
        new_src_corpus.append(old_src)
        new_tgt_corpus.append(old_tgt)
        new_src = lexical_sub(old_src, wv)
        new_tgt = lexical_sub(old_tgt, wv)
        

        if new_src: 
            new_src_corpus.append(new_src)
            new_tgt_corpus.append(old_tgt)
            
        if new_tgt: 
            new_src_corpus.append(old_src)
            new_tgt_corpus.append(new_tgt)

    return new_src_corpus, new_tgt_corpus

In [ ]:
w2v_path = '/content/drive/MyDrive/ko.bin'

from gensim.models.word2vec import Word2Vec

wv_model = Word2Vec.load(w2v_path)

In [ ]:
new_que_corpus, new_ans_corpus = augment_corpus(questions, answers, wv_model)

  0%|          | 0/11823 [00:00<?, ?it/s]

In [ ]:
sr_corpus = []; tg_corpus = []
for que_sen in new_que_corpus :
  que_sen = preprocess_sentence(que_sen)
  sr_corpus.append(que_sen)
for ans_sen in new_ans_corpus :
  ans_sen = preprocess_sentence(ans_sen)
  tg_corpus.append(ans_sen)  

In [ ]:
new_que_corpus[0]

'12시 땡!'

In [ ]:
new_ans_corpus[0]

'하루가 또 가네요.'

In [ ]:
print(len(new_que_corpus), len(new_ans_corpus))

30465 30465


In [ ]:
mecab = Mecab()
que_morph = [" ".join(mecab.morphs(sen)) for sen in new_que_corpus]
ans_morph = [" ".join(mecab.morphs(sen)) for sen in new_ans_corpus]
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',
                                                     num_words=30000)
tokenizer.fit_on_texts(que_morph)
tokenizer.fit_on_texts(ans_morph)

q_tensor = tokenizer.texts_to_sequences(que_morph)
q_tensor = tf.keras.preprocessing.sequence.pad_sequences(q_tensor, padding='post')
a_tensor = tokenizer.texts_to_sequences(ans_morph)
a_tensor = tf.keras.preprocessing.sequence.pad_sequences(a_tensor, padding='post')

In [ ]:
s = ["<start>"]; e = ["<end>"]
print(type(s),s[0])

<class 'list'> <start>


In [ ]:
attr = []
for ans_sen in new_ans_corpus :
  man = [[]for _ in range(3)]
  for i in range(3):
    man[0] = s[0]; man[1]=ans_sen; man[2] = e[0]
  attr.append(man)
a_corpus = attr    

In [ ]:
a_corpus[0]

['<start>', '하루가 또 가네요.', '<end>']

In [ ]:
np.max(a_tensor)

7266

In [ ]:
a_tensor[0]

array([378,   7, 177,   7,  59,   1,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0], dtype=int32)

### Mecab()의 토큰화와 단어 인덱싱

In [ ]:
tokenizer = Mecab()
X_train = []
for sen in new_que_corpus :
    tr_sen = tokenizer.morphs(sen) # 토큰화
    X_train.append(tr_sen)

X_test = []
for sen in new_ans_corpus :
    te_sen = tokenizer.morphs(sen) # 토큰화
    X_test.append(te_sen)
    
words = np.concatenate(X_train).tolist()
counter = Counter(words)
counter = counter.most_common(30000-3)
vocab = ['<pad>','<start>','<end>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
word_to_index["<pad>"]=0
word_to_index["<start>"]=1
word_to_index["<end>"]=2

index_to_word = {index:word for word, index in word_to_index.items()}
index_to_word[0]="<pad>"
index_to_word[1]="<start>"
index_to_word[2]="<end>"

for i in range(len(X_test)) :
  X_test[i] = ['<start>'] + X_test[i] + ['<end>']
        
def wordlist_to_indexlist(wordlist):
    return [word_to_index[word] if word in word_to_index else word_to_index['<pad>'] for word in wordlist]

def indexlist_to_wordlist(indexlist):
        return [index_to_word[index] if index in index_to_word else index_to_word['<pad>'] for index in indexlist]    
        
e_train = list(map(wordlist_to_indexlist, X_train))
d_train = list(map(wordlist_to_indexlist, X_test))
e_train = tf.keras.preprocessing.sequence.pad_sequences(e_train, maxlen=52, padding='post')
d_train = tf.keras.preprocessing.sequence.pad_sequences(d_train, maxlen=52, padding='post')

In [ ]:
np.max(d_train)

5701

In [ ]:
d_train[0]

array([   1,  319,    8,  122,    8, 1372,    7,    2,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
enc_train = e_train; dec_train = d_train

### 사용될 트랜스포머의 여러 모듈 정의

In [ ]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [ ]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [ ]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [ ]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = LearningRateScheduler(d_model=512)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

### 트랜스포머 객체 정의와 사용될 상수 정의

In [ ]:
Tran = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=30000,
    tgt_vocab_size=30000,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512

In [ ]:
learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [ ]:
print(len(new_que_corpus), len(new_ans_corpus))

30465 30465


In [ ]:
enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_train, dec_train, test_size=0.01)

print("enc_train :", len(enc_train), "enc_val :", len(enc_val))
print("dec_train :", len(dec_train), "dec_val :",len(dec_val))

enc_train : 30160 enc_val : 305
dec_train : 30160 dec_val : 305


In [ ]:
print(len(enc_train[2]))
print(len(dec_train[2]))

52
52


### 훈련 및 관찰

In [ ]:
EPOCHS = 3
BATCH_SIZE = 64

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(batch_size=BATCH_SIZE)

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)
    for step, (enc_batch, dec_batch) in enumerate(train_dataset):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_batch,
                    dec_batch,
                    Tran,
                    optimizer)

        total_loss += batch_loss
        
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
        tqdm_bar.update()

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

### 트랜스포머 모델의 번역 성능 알아보기

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=30000)

In [ ]:
def translate(tokens, model):
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=52,
                                                           padding='post')
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)   
    for i in range(52):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = indexlist_to_wordlist(ids)  
            return result

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = indexlist_to_wordlist(ids)  
    return result

print("슝=3")

슝=3


In [ ]:
def listToString(str_list):
     result = ""
     for s in str_list:
        result += s + " "
     return result.strip()

In [ ]:
def eval_bleu_single(model, src_sentence, tgt_sentence, verbose=True):

    src_tokens = tf.keras.preprocessing.text.text_to_word_sequence(src_sentence)
    tgt_tokens = tf.keras.preprocessing.text.text_to_word_sequence(tgt_sentence)

    tokens_s = wordlist_to_indexlist(src_tokens)
    tokens_t = wordlist_to_indexlist(tgt_tokens)

    if (len(tokens_s) > 52): return None
    if (len(tokens_t) > 52): return None

    reference = tgt_sentence
    candidate = listToString(translate(tokens_s, model))

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score
        
print('슝=3')

슝=3


### 번역 성능 측정하기(1) : BLEU 스코어

In [ ]:
test_idx = 0

eval_bleu_single(Tran, new_que_corpus[test_idx], new_ans_corpus[test_idx])

Source Sentence:  12시 땡!
Model Prediction:  저 랑 함께 해요 !
Real:  하루가 또 가네요.
Score: 0.024808



0.024808415001701823

In [ ]:
Q = '간만에 여자친구랑 데이트 하기로 했어.'; A = '떨리 겠 죠'
eval_bleu_single(Tran, Q, A)

Source Sentence:  간만에 여자친구랑 데이트 하기로 했어.
Model Prediction:  약 이 많 은가 봐요 .
Real:  떨리 겠 죠
Score: 0.018477



0.018476860420522198

In [ ]:
def eval_bleu(model, src_sentences, tgt_sentences, verboses=True):
    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentences[idx], verbose=False)
        if not score: continue
        
        total_score += score
    
    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)
    
print("슝=3")

슝=3


In [ ]:
dec_val[0]

array([   1,  288, 2534,  152,    3,   10,  618,   83,  152,   58,  262,
        141,   45,   16,   48,  290,  146,    7,    2,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
eval_bleu(Tran, new_que_corpus[0:2], new_ans_corpus[0:2], verboses=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Num of Sample: 2
Total Score: 0.016980478311200378


In [ ]:
reference = "12 시 땡 !".split()
candidate = "저 랑 함께 해요 !".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['12', '시', '땡', '!']
번역문: ['저', '랑', '함께', '해요', '!']
BLEU Score: 1.2183324802375697e-231


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
print("1-gram:", sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))
print("2-gram:", sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))
print("3-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))
print("4-gram:", sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))

1-gram: 0.2
2-gram: 2.2250738585072626e-308
3-gram: 2.2250738585072626e-308
4-gram: 2.2250738585072626e-308


In [ ]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.2
BLEU-2: 0.025
BLEU-3: 0.03333333333333333
BLEU-4: 0.05000000000000001

BLEU-Total: 0.05372849659117709


### 번역 성능 측정하기(2) : Beam Search Decoder

In [ ]:
def calc_prob(src_ids, tgt_ids, model):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(src_ids, tgt_ids)

    predictions, enc_attns, dec_attns, dec_enc_attns =\
    model(src_ids, 
            tgt_ids,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask)
    
    return tf.math.softmax(predictions, axis=-1)

In [ ]:
def beam_search_decoder(sentence, 
                        src_len,
                        tgt_len,
                        model,
                        beam_size):
    tokens = tf.keras.preprocessing.text.text_to_word_sequence(sentence)
    tokens = wordlist_to_indexlist(tokens)
    
    src_in = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                            maxlen=52,
                                                            padding='post')

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.int64)
    pred_tmp = np.zeros((beam_size, tgt_len), dtype=np.int64)

    eos_flag = np.zeros((beam_size, ), dtype=np.int64)
    scores = np.ones((beam_size, ))

    pred_tmp[:, 0] = word_to_index["<start>"]

    dec_in = tf.expand_dims(pred_tmp[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size, ))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            score_cache[cache_pos:cache_pos+beam_size] = scores[branch_idx]
            pred_cache[cache_pos:cache_pos+beam_size, :seq_pos] = \
            pred_tmp[branch_idx, :seq_pos]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            if seq_pos != 1:   # 모든 Branch를 로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos+beam_idx] *= prob[max_idx]
                pred_cache[cache_pos+beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1: continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, :seq_pos+1]

            pred_tmp[beam_idx, :seq_pos+1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == word_to_index["<end>"]:
                eos_flag[beam_idx] = -1

    pred = []
    for long_pred in pred_tmp:
        zero_idx = long_pred.tolist().index(word_to_index["<end>"])
        short_pred = long_pred[:zero_idx+1]
        pred.append(short_pred)
    return pred

In [ ]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                            candidate,
                            weights=weights,
                            smoothing_function=SmoothingFunction().method1)

print('슝=3')

슝=3


In [ ]:
def beam_bleu(reference, ids):

    total_score = 0.0
    for _id in ids:
        candidate = indexlist_to_wordlist(_id.tolist())
        candidate = listToString(candidate)
        score = calculate_bleu(reference, candidate)

        print("Reference:", reference)
        print("Candidate:", candidate)
        print("BLEU:", calculate_bleu(reference, candidate))

In [ ]:
# 인덱스를 바꿔가며 확인해 보세요
test_idx = 2

ids = \
beam_search_decoder(new_que_corpus[test_idx],
                    52,
                    52,
                    Tran,
                    beam_size=5)

bleu = beam_bleu(new_ans_corpus[test_idx], ids)
print(bleu)

Reference: 위로해 드립니다.
Candidate: <start> 좋 은 생 님 이 네 요 . <end>
BLEU: 0.007696339439306161
Reference: 위로해 드립니다.
Candidate: <start> 좋 은 생 님 이 되 요 . <end>
BLEU: 0.007696339439306161
Reference: 위로해 드립니다.
Candidate: <start> 좋 은 생 님 이 좋 요 . <end>
BLEU: 0.007696339439306161
Reference: 위로해 드립니다.
Candidate: <start> 좋 은 생 님 이 님 요 . <end>
BLEU: 0.007696339439306161
Reference: 위로해 드립니다.
Candidate: <start> 좋 은 생 님 을 네 요 . <end>
BLEU: 0.007696339439306161
None


<회고>

* 처음에는 번역기와 챗봇이 같은 집안이라는 글쓴이의 말씀이 잘 이해되지   않았습니다. \\
그러나 트랜스포머라는 구조도 결국은 문장을 숫자로 인코딩하여 숫자를 다시 문장으로 디코딩하는 과정이므로 두 경우에 모두 사용될 수 있음은 이해했습니다. \\


* BLEU 점수를 산정하는 방법은 합이 아닌 곱에 대한 경우라서 지수에 각각 가중치를 부여한다는 것을 알겠습니다. 그리고 각 n-gram에 대해 다양한 점수가 나온다는 점이 흥미롭네요.

* Beam Search Decoder를 이용한 문장 해석 방법은 원리로는 이해가 가나 코드로 이해하기에는 시간이 필요한 것 같습니다. 각 스텝마다 단어를  beam 사이즈 만큼 준비해서 각각의 확률을 구하고 또 다시 각 단어에 대해서 beam 사이즈 만큼의 단어를 구해 확률을 구하는 과정을 반복하는 원리가 기발한 것 같습니다. 

*   과제작성 중 어려웠던 점 :

   1. 답변 문장 양끝에 ['start'] 토큰리스트와 ['end'] 토큰리스트를 붙이는 과정에서 컴퓨터가 계속 토큰을 리스트가 아닌 스트링으로 인식하여 고전했습니다. 결국 한국어 토큰화 도구인 Mecab으로 직접 어휘사전을 만들고 인덱싱을 하고 word_to_index, index_to_word 등등 함수를 일일히 만들어야 했습니다.
   2.   거의 마지막에 해석을 만드는 함수를 수정하는 과정도 만만치 않았습니다. 입력 토크나이저와 출력 토크나이저를 같게 놓거나 또는 필요없게 만듦으로써 문제를 부분적으로 해결하였으나 나머지는 1에서 말한 여러 함수로 대체해야 했습니다.

* 아무튼 과제를 완성하게 되었으니 그걸로 다행입니다.
* 시간이 난다면 여러 언어나 LSTM과 같은 모델을 시도해보고 싶네요. 

* 번역결과 \\
Source Sentence:  간만에 여자친구랑 데이트 하기로 했어. \\
Model Prediction:  약 이 많 은가 봐요 . \\
Real:  떨리 겠 죠 \\
Score: 0.018477 \\
 0.018476860420522198 \\

* 사용된 상수들 : \\

1.  Transformer : \\
n_layers=2, \\
d_model=512, \\
n_heads=8, \\
d_ff=2048, \\
src_vocab_size=30000, \\
tgt_vocab_size=30000, \\
pos_len=200, \\
dropout=0.3, \\
shared_fc=True, \\
shared_emb=True \\
d_model = 512 \\

2.  optimizer : \\
beta_1=0.9, beta_2=0.98, epsilon=1e-9 \\

3.  Training :

 EPOCHS : 3 \\
BATCH_SIZE : 64 \\
warmup_steps : 4000 \\





